In [2]:
import pandas as pd
from models import ElectricityRecord

In [3]:
# text file paths
area_file_path = 'Area.txt'
date_dim_file_path = 'DateDim.txt'
dwelling_file_path = 'Dwelling.txt'
electricity_file_path = 'Electricity.txt'

# Approach 1: Pandas

In [4]:
area_df = pd.read_csv(area_file_path, sep=';')
date_dim_df = pd.read_csv(date_dim_file_path, sep=';')
dwelling_df = pd.read_csv(dwelling_file_path, sep=',')
electricity_df = pd.read_csv(electricity_file_path, sep=';')

## exploration
area_df.head()
# date_dim_df.head()
dwelling_df.head()
electricity_df.head()

,DateID,AreaID,dwelling_type_id,kwh_per_acc
0,54950,1001,1,86.7
1,54950,1002,1,107.5
2,54950,1003,1,113.9
3,54950,1004,1,118.1
4,54950,1005,1,123.2


In [5]:
# merging DataFrames using Pandas library functions
merged_df = pd.merge(electricity_df, area_df, on='AreaID', how='inner')
merged_df = pd.merge(merged_df, date_dim_df, on='DateID', how='inner')
merged_df = pd.merge(merged_df, dwelling_df, left_on='dwelling_type_id', right_on='TypeID', how='inner')

merged_df.head()

,DateID,AreaID,dwelling_type_id,kwh_per_acc,Area,Region,year,month,quarter,TypeID,dwelling_type
0,54950,1001,1,86.7,Bishan,Central Region,2010,1,1,1,1-room / 2-room
1,54950,1002,1,107.5,Sembawang,North Region,2010,1,1,1,1-room / 2-room
2,54950,1003,1,113.9,Yishun,North Region,2010,1,1,1,1-room / 2-room
3,54950,1004,1,118.1,Outram,Central Region,2010,1,1,1,1-room / 2-room
4,54950,1005,1,123.2,Kallang,Central Region,2010,1,1,1,1-room / 2-room


In [6]:
# calculate average kwh_per_acc per dwelling type in each region, year and month
denormalized_df = merged_df.groupby(['Region', 'Area', 'year', 'month', 'dwelling_type'])['kwh_per_acc'].mean().reset_index()
denormalized_df = denormalized_df.rename(columns={'kwh_per_acc': 'avg_kwh_per_acc'})
denormalized_df['avg_kwh_per_acc'] = denormalized_df['avg_kwh_per_acc'].round(2)
denormalized_df = denormalized_df.sort_values(['Area', 'year', 'month', 'dwelling_type'])

# calculate average kwh_per_acc per dwelling type in each region, year and month

# uncomment to format this column to 2dp to look like the output, but will turn into a object type
# denormalized_df['avg_kwh_per_acc'] = denormalized_df['avg_kwh_per_acc'].apply(lambda x: '{:,.2f}'.format(x)) # source: https://stackoverflow.com/questions/57194853/how-to-read-this-lambda-x-fx06

# confirm similarity to example output
denormalized_df.head(20)

# check types, object - string
denormalized_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26925 entries, 14251 to 22087
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Region           26925 non-null  object 
 1   Area             26925 non-null  object 
 2   year             26925 non-null  int64  
 3   month            26925 non-null  int64  
 4   dwelling_type    26925 non-null  object 
 5   avg_kwh_per_acc  26925 non-null  float64
dtypes: float64(1), int64(2), object(3)
memory usage: 1.4+ MB


In [7]:
# Instructions:
# The final dataset generated should be list of objects (a must to test your OOP concepts)
records = [
        ElectricityRecord(
            region=row.Region,
            area=row.Area,
            year=row.year,
            month=row.month,
            dwelling_type=row.dwelling_type,
            avg_kwh_per_acc=row.avg_kwh_per_acc
        )
        for row in denormalized_df.itertuples() # itertuples is faster than iterrows
    ]

# explore result
print(f"Number of records: {len(records)}")
print("\nFirst 3 records:")
for record in records[:3]:
    print(record)

Number of records: 26925

First 3 records:
ElectricityRecord(region='North East Region', area='Ang Mo Kio', year=2010, month=1, dwelling_type='1-room / 2-room', avg_kwh_per_acc=144.9)
ElectricityRecord(region='North East Region', area='Ang Mo Kio', year=2010, month=1, dwelling_type='Landed Properties', avg_kwh_per_acc=785.2)
ElectricityRecord(region='North East Region', area='Ang Mo Kio', year=2010, month=1, dwelling_type='Private Apartments and Condominiums', avg_kwh_per_acc=445.0)


# Non-Pandas Approach

Without using Pandas DataFrames, we can treat each row of data as a dictionary

In [8]:
area_data = []
with open('Area.txt', 'r') as file:
    # skip the header row
    next(file) 
    for line in file:
        area_id, area, region = line.strip().split(';')
        datapoint = {}
        datapoint['area_id'] = int(area_id)
        datapoint['area'] = area
        datapoint['region'] = region
        area_data.append(datapoint)
        
date_dim_data = []
with open('DateDim.txt', 'r') as file: 
    # skip the header row
    next(file)
    for line in file:
        date_id, year, month, quarter = line.strip().split(';')
        datapoint = {}
        datapoint['date_id'] = int(date_id)
        datapoint['year'] = year
        datapoint['month'] = month
        datapoint['quarter'] = quarter
        date_dim_data.append(datapoint)

dwelling_data = []
with open('Dwelling.txt', 'r') as file:
    # skip the header row
    next(file)
    for line in file:
        type_id, dwelling_type = line.strip().split(',')
        datapoint = {}
        datapoint['type_id'] = int(type_id)
        datapoint['dwelling_type'] = dwelling_type
        dwelling_data.append(datapoint)

electricity_data = []
with open('Electricity.txt', 'r') as file:
    # skip the header row
    next(file)
    for line in file:
        date_id, area_id, dwelling_type_id, kwh_per_acc = line.strip().split(';')
        datapoint = {}
        datapoint['date_id'] = int(date_id)
        datapoint['area_id'] = int(area_id)
        datapoint['type_id'] = int(dwelling_type_id)
        datapoint['kwh_per_acc'] = float(kwh_per_acc)
        electricity_data.append(datapoint)

In [9]:
# merging data dictionaries
# credits: https://stackoverflow.com/a/5946322

merged_data = []

for electricity_record in electricity_data:
    for area in area_data:
        if electricity_record['area_id'] == area['area_id']:
            merged_record = {**electricity_record, **area}
            merged_data.append(merged_record)

date_dict = {record['date_id']: record for record in date_dim_data} # O(n)
for i in range(len(merged_data)): # O(m)
    date_id = merged_data[i]['date_id'] # O(1)
    if date_id in date_dict:
        merged_data[i] = {**date_dict[date_id], **merged_data[i]} 


dwelling_dict = {record['type_id']: record for record in dwelling_data}
for i in range(len(merged_data)):
    type_id = merged_data[i]['type_id']
    if type_id in dwelling_dict:
        merged_data[i] = {**dwelling_dict[type_id], **merged_data[i]}

# explore result
print(merged_data[:3])


[{'type_id': 1, 'dwelling_type': '1-room / 2-room', 'date_id': 54950, 'year': '2010', 'month': '1', 'quarter': '1', 'area_id': 1001, 'kwh_per_acc': 86.7, 'area': 'Bishan', 'region': 'Central Region'}, {'type_id': 1, 'dwelling_type': '1-room / 2-room', 'date_id': 54950, 'year': '2010', 'month': '1', 'quarter': '1', 'area_id': 1002, 'kwh_per_acc': 107.5, 'area': 'Sembawang', 'region': 'North Region'}, {'type_id': 1, 'dwelling_type': '1-room / 2-room', 'date_id': 54950, 'year': '2010', 'month': '1', 'quarter': '1', 'area_id': 1003, 'kwh_per_acc': 113.9, 'area': 'Yishun', 'region': 'North Region'}]


In [10]:
# calculate average
from collections import defaultdict
from statistics import mean

# store grouped values
grouped_data = defaultdict(list)

# group the data
for record in merged_data:
    group_key = (
        record['region'],
        record['area'],
        record['year'],
        record['month'],
        record['dwelling_type']
    )
    grouped_data[group_key].append(record['kwh_per_acc'])

denormalized_data = [
    {
        'region': region,
        'area': area,
        'year': year,
        'month': month,
        'dwelling_type': dwelling_type,
        'avg_kwh_per_acc': mean(values)
    }
    for (region, area, year, month, dwelling_type), values in grouped_data.items()
]

# sort the result
denormalized_data.sort(key=lambda x: (
    x['area'],
    x['year'],
    x['month'],
    x['dwelling_type']
))

# explore result
print(denormalized_data[:3])

[{'region': 'North East Region', 'area': 'Ang Mo Kio', 'year': '2010', 'month': '1', 'dwelling_type': '1-room / 2-room', 'avg_kwh_per_acc': 144.9}, {'region': 'North East Region', 'area': 'Ang Mo Kio', 'year': '2010', 'month': '1', 'dwelling_type': 'Landed Properties', 'avg_kwh_per_acc': 785.2}, {'region': 'North East Region', 'area': 'Ang Mo Kio', 'year': '2010', 'month': '1', 'dwelling_type': 'Private Apartments and Condominiums', 'avg_kwh_per_acc': 445.0}]


In [11]:
# generate list of objects as per requirements
final_records = [ElectricityRecord(record['region'], record['area'], record['year'], record['month'], record['dwelling_type'], record['avg_kwh_per_acc']) for record in denormalized_data]

# explore result
print(f"Number of records: {len(final_records)}")
print("\nFirst 3 records:")
for record in final_records[:3]:
    print(record)

Number of records: 26925

First 3 records:
ElectricityRecord(region='North East Region', area='Ang Mo Kio', year=2010, month=1, dwelling_type='1-room / 2-room', avg_kwh_per_acc=144.9)
ElectricityRecord(region='North East Region', area='Ang Mo Kio', year=2010, month=1, dwelling_type='Landed Properties', avg_kwh_per_acc=785.2)
ElectricityRecord(region='North East Region', area='Ang Mo Kio', year=2010, month=1, dwelling_type='Private Apartments and Condominiums', avg_kwh_per_acc=445.0)


# Miscelleanous 

In [24]:
unique_regions = {}
index = 1

for area in area_data:
    if area['region'] not in unique_regions.values():
        unique_regions[index] = area['region']
        index += 1

# to generate the choices in query.py
items = []
for key, val in unique_regions.items():
    formatted_item = f"{key}: {val}"
    padded_item = formatted_item.ljust(25)
    items.append(padded_item)

rows = []
for i in range(0, len(items), 3):
    row = items[i:i + 3]
    rows.append("        " + "".join(row))

print("\n".join(rows))

        1: Central Region        2: North Region          3: West Region           
        4: North East Region     5: East Region           


In [18]:
unique_dwelling_type = {}

for dwelling in dwelling_data:
    if dwelling['dwelling_type'] not in unique_dwelling_type:
        unique_dwelling_type[dwelling['type_id']] = dwelling['dwelling_type']

for key, val in unique_dwelling_type.items():
    print(f"{key}: {val}")

1: 1-room / 2-room
2: Private Apartments and Condominiums
3: Landed Properties
4: 5-room and Executive
5: 3-room
6: 4-room


In [ ]:
# group records by area over month
area_groupings = {}
for record in records:
if record['area'] not in area_groupings:
    area_groupings['area'] = {}
area_records[record.area][record.month] = record.avg_kwh_per_acc

for month in range(1, 13):
monthly_usage_sum = 0
for record in records:
    if record['month'] == month:
        monthly_usage_sum += record['avg_kwh_per_acc'] 
        # ...realized not necessary, because there is only one avg_kwh_per_acc per month, area and dwelling type anyway as a result of task 1, can direct assign on area_groupings

In [27]:
# exploratory manual testing
from query import find_matching_records

find_matching_records(final_records, 2010, 'North East Region', '3-room')

Found 0 matching records.


[]

# Presentation

**For task (i),** 

- **explain how you apply divide and conquer method to this problem, what are the algorithms you 
have proposed**

I broke down the task of merging and denomalizing the datasets into 4 separate functions found in `complete.py.` The 4 functions are:
    
- `read-input` which reads the .txt files with the original data as Pandas DataFrames
- `merge` which merges the 4 DataFrames over common columns
- `transform` which filters the dataset to the relevant columns (denormalizes by removing id columns)
- `make_output` which creates ElectricityRecord objects representing each row of data and returns them all in a list as per the task requirements
 

- **illustrate how you perform Time complexity analysis.**

The time complexity code is found in `timing.py`. The same 4 functions were coded in two different ways, one using Pandas and one without. Splitting up the functions this way allowed me to test and time each one separately.

The most important timing metric is **merging**. It is much faster to merge using Pandas.

- **discuss how the final selection of algorithm(s) is done**

I picked the approach using Pandas DataFrames as my timing tests consistently showed it had faster results than the pure Python approach managing my own dictionaries and lists. Sometimes, it was up to 4x faster.

- **demo the execution of final.py and show case the dataset required is generated.**

Shown in the video, or you may run `python .\query.py` in the command line to interact with the program.

**For task (ii),**

- **explain how you design the object and data structure used for this program**

My ElectricityRecord class is located in `models.py` and its constructor takes the region, area, year, month, dwelling type and average kWh per acc for that time period and locale. I designed the class in this way to match the expected output as shown in the assignment instructions and to make it simple to filter for matching ElectricityRecords based on year, dwelling type and region in task (ii), because these properties can be accessed and checked with dot notation.

- **demo how you perform the query with various filtering.**

Shown in the video.

- **illustrate the major input validation**

Shown in the video, but some manual testings of edge and special cases are entering non-numeric input or input outside of the expected range.

**Include your learning experience and journey. 
If you have used generative AI to assist in completing this assignment, you are required to include why 
is there a need of use and how have you improved after the use.**

I used generative AI in two main aspects:
1. To help me refactor my timing tests in `timing.py` to reuse it multiple times for each function and approach. 
2. To help me format the final output for task (ii), which was a large ASCII-like table.

I first tried to do it on my own by refering to the timing tests for Binary Search in class, and some of the text formatting examples from Trimester 1. However, it was challenging because I kept running into errors with the formatting and was unsure of how to stucture the tests for optimal reusability. I could not properly visualize how to make the right formatting for the table and would have been lost without the AI's help.

One interesting aspect of my learning journey is that when I wrote the `complete.py` file, I acidentally put in all the code for the pure Python approach when I intended to use the Pandas approach. I then wrote all of the code for task (ii) using this wrong version of `complete.py`. When I discovered my mistake, I was pleased that I didn't have to change much at all besides replacing `complete.py` with my Pandas code. This is because both codes return the same thing: a list of ElectricityRecords, so my query program in `query.py` is nicely decoupled and agnostic to the internal workings of both codes!

One thing I think did well is modularizing my code using a divide-and-conquer approach to break it up into smaller functions that are responsible for separate things, like `merge` and `transform`. This allowed me to test different parts of my code easily and improved its readabilty.

One thing I would improve is not hardcoding the size inputs in the test files, since I made the assumption that the data is static because it is historical from the past 10 years and will never change. I think an ideal solution would be more robust to indexing errors and different inputs.